# Handling Missing Data
Real world data is rarely clean and homogeneous. Many datasets will have some amount of data missing. Missing data: null, NaN, NA.

# Trade-Offs in Missing Data Conventions
We either use a mask that globally indicates missing values, or choose a senitel value that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing interger value with -9999 or some rare bit pattern, or NaN.
Use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation
A sentinel value reduces the range of valid values that can be represented and may require extra logic in CPU and GPU arithmetic
# Missing Data in Pandas
Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point NaN value, and the Python None object. This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.

## `None` Pythonic missing data
The first sentinel value used by Pandas is `None`, a python singleton object that is often used for missing data in Python code. `None` cannot be used in any arbitrary NumPy/Pandas array, but only arrays with data type `object`

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types

## `NaN` Missing numerical data
Not a number. Recognized by all systems that use the standard IEEE floating-point repesentation:

In [3]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

In [4]:
1 + np.nan

nan

In [5]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

In [6]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

**Keep in mind that NaN is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.**

In [9]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

# Operating on Null Values
## Detecting null values

In [10]:
data = pd.Series([1, np.nan, 'hello', None])

In [11]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [12]:
data[data.notnull()]

0        1
2    hello
dtype: object

## Dropping null values

In [13]:
data.dropna()

0        1
2    hello
dtype: object

In [15]:
df = pd.DataFrame([[1, np.nan, 2],
                  [2,      3, 5],
                  [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [16]:
# drop all rows which any null values are present
df.dropna()

,0,1,2
1,2.0,3.0,5


In [18]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [20]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [22]:
df.dropna(axis=1, how='all') # drop rows and columns that are all NaN

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [23]:
# thresh: minimum number of non-null values for the row/column to be kept
df.dropna(axis=0, thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


## Filling null values

In [24]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [25]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [26]:
# forward fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [27]:
# back fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [28]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [29]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
